In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
INFILE = 'master_metadata.csv'
OUTPATH = './'

In [3]:
meta_df = pd.read_csv(INFILE)

In [4]:
meta_df

,id,Title,Batch,Transcript,Allocation,Content,biografy,vimeo_promo_id,vimeo_description,Service Types,Project Types,Tags,related_videos,Priority Words
0,2643.0,Admiral William O'Brien,1,1,Train/Eval,NaN,"A remarkable interview, full of detail and opi...",116177529.0,"On PQ 17: ""I can remember vividly the scene at...",Navy,The Veterans' video archive|Keeping Britain Af...,Places>North Africa|Role>First Lieutenant|Role...,"a:14:{i:0;s:4:""2647"";i:1;s:4:""2648"";i:2;s:4:""2...","['North Africa', 'First Lieutenant', 'Navigato..."
1,2071.0,Alex Owens,1,1,Test,NaN,The delightful Alex Owens provides a classic s...,90101380.0,NaN,Navy,Keeping Britain Afloat|The Veterans' video arc...,Places>England - HMS Ganges (Stone Frigate)|Mi...,"a:7:{i:0;s:4:""2074"";i:1;s:4:""2075"";i:2;s:4:""20...","['England', 'HMS Ganges (Stone Frigate)', 'Des..."
2,2301.0,Cornelius Snelling,1,1,Train/Eval,NaN,Cornelius Snelling served on the Black Swan-cl...,93458542.0,Cornelius Snelling served on the Black Swan-cl...,Navy,The Normandy Campaign|The Veterans' video arch...,Miscellaneous>Naval - Rum Ration|Battles>Opera...,"a:12:{i:0;s:4:""2304"";i:1;s:4:""2305"";i:2;s:4:""2...","['Naval', 'Rum Ration', 'Operation', 'Neptune ..."
3,3238.0,David Craig,1,1,Train/Eval,NaN,David's interview isn't the easiest to listen ...,121183947.0,David Craig provides a fantastic account of hi...,Civilian,The Veterans' video archive|Keeping Britain Af...,Places>Russia|Miscellaneous>Naval Convoy - JW5...,"a:8:{i:0;s:4:""3241"";i:1;s:4:""3242"";i:2;s:4:""32...","['Russia', 'Naval Convoy', 'JW53', 'Ship', 'Ru..."
4,2146.0,Dennis Whitehead,1,1,Train/Eval,NaN,Dennis Whitehead served on the C-Class Destroy...,90098682.0,It’s hard to determine if it was good or bad l...,Navy,Keeping Britain Afloat|The Veterans' video arc...,Service Type>Navy|Role>Ordinary Seaman|Places>...,"a:6:{i:0;s:4:""2149"";i:1;s:4:""2150"";i:2;s:4:""21...","['Navy', 'Ordinary Seaman', 'Mil Camp Uk', 'RN..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,2594.0,William Harry Pullen,-1,0,Train/Eval,NaN,NaN,NaN,NaN,Army,The Great War recording,NaN,NaN,"['William', 'Harry', 'Pullen']"
679,1593.0,William Seldon,-1,0,Train/Eval,NaN,NaN,75772917.0,Bill Sheldon was one of a number of Naval Vete...,Navy,The Veterans' video archive,Miscellaneous>Description - Camp / Accom|Misce...,"a:6:{i:0;s:4:""1596"";i:1;s:4:""1597"";i:2;s:4:""15...","['Description', 'Camp / Accom', 'Co', 'Short B..."
680,1116.0,William Walker,-1,0,Train/Eval,NaN,William Walker was the oldest surviving vetera...,52612037.0,William Walker was forced to bale out over the...,RAF,The Veterans' video archive,Places>RAF Brize Norton|Places>RAF Halton|Plac...,"a:4:{i:0;s:4:""1119"";i:1;s:4:""1120"";i:2;s:4:""11...","['RAF Brize Norton', 'RAF Halton', 'RAF Kenley..."
681,5703.0,Yvonne Baseden,-1,0,Train/Eval,NaN,\nYvonne started her military career with the...,146240571.0,Yvonne was a wireless operator who parachuted ...,RAF,The Veterans' video archive|The Secret War,Miscellaneous>POW|Regiment / Squadron>SOE - Sp...,"a:8:{i:0;s:4:""5707"";i:1;s:4:""5708"";i:2;s:4:""57...","['POW', 'SOE', 'Special Operations Executive',..."


In [5]:
# Identify key items from tags, interviewee name

def get_priority_items(inrow):
    
    patt_service_loc = re.compile(r"[A-Z]+\s(.+)")
    
    ww = []

    if type(inrow.Tags) == str:
        # Split taglist on pipes, then into sub-tags
        for f in inrow.Tags.split('|'):
            ww.append(re.split(' - ',re.split('[>]',f)[-1]))

        # Flatten split lists
        ww = [item.strip() for sublist in ww for item in sublist]

        # Extract shortned names from likely service locations, e.g. for "RAF Halesworth" we also append "Halesworth"
        for w in ww:
            loc_match = re.fullmatch(patt_service_loc,w)
            if loc_match:
                ww.append(loc_match.group(1))
            
    return ww

In [6]:
def get_name_words(inrow):
    
    ww = []
    
    # Finally add each of the interviewee's names
    for n in inrow.Title.split(" "):
        # Strip quote marks from nicknames
        n = n.strip(" \"'")
        if len(n): ww.append(n)
            
    return ww

In [8]:
meta_df['Priority Words'] = meta_df.apply(get_priority_items, axis=1)
meta_df['Name Words'] = meta_df.apply(get_name_words, axis=1)
meta_df

,id,Title,Batch,Transcript,Allocation,Content,biografy,vimeo_promo_id,vimeo_description,Service Types,Project Types,Tags,related_videos,Priority Words,Name Words
0,2643.0,Admiral William O'Brien,1,1,Train/Eval,NaN,"A remarkable interview, full of detail and opi...",116177529.0,"On PQ 17: ""I can remember vividly the scene at...",Navy,The Veterans' video archive|Keeping Britain Af...,Places>North Africa|Role>First Lieutenant|Role...,"a:14:{i:0;s:4:""2647"";i:1;s:4:""2648"";i:2;s:4:""2...","[North Africa, First Lieutenant, Navigator, WW...","[Admiral, William, O'Brien]"
1,2071.0,Alex Owens,1,1,Test,NaN,The delightful Alex Owens provides a classic s...,90101380.0,NaN,Navy,Keeping Britain Afloat|The Veterans' video arc...,Places>England - HMS Ganges (Stone Frigate)|Mi...,"a:7:{i:0;s:4:""2074"";i:1;s:4:""2075"";i:2;s:4:""20...","[England, HMS Ganges (Stone Frigate), Descript...","[Alex, Owens]"
2,2301.0,Cornelius Snelling,1,1,Train/Eval,NaN,Cornelius Snelling served on the Black Swan-cl...,93458542.0,Cornelius Snelling served on the Black Swan-cl...,Navy,The Normandy Campaign|The Veterans' video arch...,Miscellaneous>Naval - Rum Ration|Battles>Opera...,"a:12:{i:0;s:4:""2304"";i:1;s:4:""2305"";i:2;s:4:""2...","[Naval, Rum Ration, Operation, Neptune / Overl...","[Cornelius, Snelling]"
3,3238.0,David Craig,1,1,Train/Eval,NaN,David's interview isn't the easiest to listen ...,121183947.0,David Craig provides a fantastic account of hi...,Civilian,The Veterans' video archive|Keeping Britain Af...,Places>Russia|Miscellaneous>Naval Convoy - JW5...,"a:8:{i:0;s:4:""3241"";i:1;s:4:""3242"";i:2;s:4:""32...","[Russia, Naval Convoy, JW53, Ship, Russian / A...","[David, Craig]"
4,2146.0,Dennis Whitehead,1,1,Train/Eval,NaN,Dennis Whitehead served on the C-Class Destroy...,90098682.0,It’s hard to determine if it was good or bad l...,Navy,Keeping Britain Afloat|The Veterans' video arc...,Service Type>Navy|Role>Ordinary Seaman|Places>...,"a:6:{i:0;s:4:""2149"";i:1;s:4:""2150"";i:2;s:4:""21...","[Navy, Ordinary Seaman, Mil Camp Uk, RNB Chath...","[Dennis, Whitehead]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,2594.0,William Harry Pullen,-1,0,Train/Eval,NaN,NaN,NaN,NaN,Army,The Great War recording,NaN,NaN,[],"[William, Harry, Pullen]"
679,1593.0,William Seldon,-1,0,Train/Eval,NaN,NaN,75772917.0,Bill Sheldon was one of a number of Naval Vete...,Navy,The Veterans' video archive,Miscellaneous>Description - Camp / Accom|Misce...,"a:6:{i:0;s:4:""1596"";i:1;s:4:""1597"";i:2;s:4:""15...","[Description, Camp / Accom, Co, Short Brothers...","[William, Seldon]"
680,1116.0,William Walker,-1,0,Train/Eval,NaN,William Walker was the oldest surviving vetera...,52612037.0,William Walker was forced to bale out over the...,RAF,The Veterans' video archive,Places>RAF Brize Norton|Places>RAF Halton|Plac...,"a:4:{i:0;s:4:""1119"";i:1;s:4:""1120"";i:2;s:4:""11...","[RAF Brize Norton, RAF Halton, RAF Kenley, RAF...","[William, Walker]"
681,5703.0,Yvonne Baseden,-1,0,Train/Eval,NaN,\nYvonne started her military career with the...,146240571.0,Yvonne was a wireless operator who parachuted ...,RAF,The Veterans' video archive|The Secret War,Miscellaneous>POW|Regiment / Squadron>SOE - Sp...,"a:8:{i:0;s:4:""5707"";i:1;s:4:""5708"";i:2;s:4:""57...","[POW, SOE, Special Operations Executive, Train...","[Yvonne, Baseden]"


In [9]:
meta_df.to_csv(INFILE, index=False)

In [7]:
# Subset by batches, select specified number (of interviews with transcripts) for allocation to test set if not already done

batch_allocation = {-1 : 0,
                     0 : 0,
                     1 : 8}

for b,test_target in batch_allocation.items():
    
    # Get unallocated items from this batch
    filt = (meta_df.Batch == b) & (pd.isnull(meta_df.Allocation))
    b_df = meta_df[filt]
    
    # If none selected, do nothing - all allocated already
    if len(b_df) == 0:
        print("Batch {} has no unallocated items. Skipping.".format(str(b)))
        pass
    
    else:
        # Look for entries with transcripts, check there are enough
        b_gold = b_df[b_df.Transcript > 0]
        
        if len(b_gold) < test_target:
            raise ValueError("Batch {} has {} items with gold standard, but {} requested. Aborting.".format(str(b),len(b_gold),test_target))
            
        elif test_target < 0:
            raise ValueError("Target count must be >= 0, but {} requested. Aborting.".format(test_target))
            
        elif test_target == 0:
            # Nothing to go to test
            b_df = b_df.assign(Allocation = 'Train/Eval')
            meta_df.update(b_df[['Allocation']])
            
            meta_df.to_csv(INFILE, index=False)
                        
        else:
            b_to_train = b_df.copy()
            
            b_to_test  = b_gold.sample(test_target,random_state = 466741)
                        
            b_to_train = pd.merge(b_df, b_to_test, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
            
            b_to_test = b_to_test.assign(Allocation = 'Test')
            b_to_train = b_to_train.assign(Allocation = 'Train/Eval')
            
            meta_df.update(b_to_test[['Allocation']])
            meta_df.update(b_to_train[['Allocation']])
            
            b_to_test.to_csv(OUTPATH+'batch_{}_test.csv'.format(b),index=False)
            b_to_train.to_csv(OUTPATH+'batch_{}_train.csv'.format(b),index=False)
            meta_df.to_csv(INFILE, index=False)
        

Batch -1 has no unallocated items. Skipping.
Batch 0 has no unallocated items. Skipping.
Batch 1 has no unallocated items. Skipping.


In [10]:
meta_df.Allocation.value_counts()

Train/Eval    675
Test            8
Name: Allocation, dtype: int64